In [5]:
import wget
import pandas as pd
import os
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
import plotly
filename = wget.download(
    'https://raw.githubusercontent.com/'
    'CSSEGISandData/COVID-19/master/csse_covid_19_data/'
    'csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
)
df1 = pd.read_csv(filename)
os.remove(filename)

filename = wget.download(
    'https://raw.githubusercontent.com/'
    'COVID19Tracking/covid-tracking-data/master/data/states_daily_4pm_et.csv'
)
df2 = pd.read_csv(filename)
os.remove(filename)

hex_colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [2]:
def plot_global_cases(dfAll):
    dates = dfAll.columns[4:]
    dfCountry = dfAll[dfAll['Province/State'].isna()]
    dfCountry = dfCountry.drop(columns=['Province/State', 'Lat', 'Long'])
    dfCountry = dfCountry.T
    dfCountry.columns = dfCountry.iloc[0]
    dfCountry = dfCountry.drop(['Country/Region'])
    dfCountry = dfCountry.assign(date = dates)
    
    
    country_with_state = dfAll[~dfAll['Province/State'].isna()]['Country/Region'].unique()
    dftmp = dfAll.drop(columns=['Province/State', 'Country/Region', 'Lat', 'Long'])
    
    countries = dict()
    for country in country_with_state:
        df = dfAll[dfAll['Country/Region']==country]
        df = df.drop(columns=['Province/State', 'Country/Region', 'Lat', 'Long'])
        row_total = None
        for idx, row in df.iterrows():
            if row_total is None:
                row_total = row
                continue
            row_total = row_total + row
        dfCountry.insert(0, country, list(row_total), True)
    
    dfCountry = dfCountry.rename(columns={"Country/Region": "date"})
    dfCountry= dfCountry.reset_index(drop=True)
    count_names = []
    for count, name in zip(list(dfCountry.loc[len(dfCountry)-1]), list(dfCountry.columns)):
        if type(count) is str:
            continue
        count_names.append((count, name))
        
    count_names.sort(key = lambda x: x[0], reverse=True)
    
    
    
    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.05)
    for idx, country in count_names:
        if country == 'date':
            continue
            
        date = dfCountry['date']
        count = dfCountry[country]
        color = hex_colors[idx % len(hex_colors)]
        last_count = list(count)[-1]
    
        fig.add_trace(
            go.Scatter(
                x=date, y=count, mode='lines+markers', name=f'{country} accumulated: last day = {last_count}', legendgroup=country,
                line=dict(color=color), marker=dict(color=color),
                hoverinfo='text',
                hovertext=[f'{country}, {t}={c}' for t, c in zip(dfCountry['date'], dfCountry[country])]
            ),
            row=1, col=1
        )
        
        count = list(dfCountry[country])
        count_diff = np.array(count) - np.array([0]+count[:-1])
        fig.add_trace(
            go.Scatter(
                x=date, y=count_diff , mode='lines+markers', name=f'{country} daily', legendgroup=country,
                line=dict(color=color), marker=dict(color=color),
                hoverinfo='text',
                hovertext=[f'{country}, {t}, daily new case={c}' for t, c in zip(date, count_diff)]
            ),
            row=2, col=1
        )
        
    fig.update_layout( title="Corona virus growth: Global")
    fig.update_yaxes(title_text="Accumulative count", row=1, col=1)
    fig.update_yaxes(title_text="Daily count", row=2, col=1)
    
    plotly.offline.plot(fig, filename = 'corona_global.html', auto_open=True)
    
plot_global_cases(df1)

In [3]:
def plot_usa_cases(dfAll):
    df = dfAll
    dfUS = df[df['Country/Region']=='US']
    states = dfUS['Province/State'].unique()
    
    dfUS = dfUS.T
    dfUS.columns = dfUS.iloc[0]
    dfUS = dfUS.drop(['Country/Region', 'Lat', 'Long', 'Province/State'])
    last_date = dfUS.index[-1]
    
    count_names = []
    for count, name in zip(list(dfUS.loc[last_date]), list(dfUS.columns)):
        if type(count) is str:
            continue
        count_names.append((count, name))
    count_names.sort(key = lambda x: x[0], reverse=True)
    
    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.1)
    for last_count, name in count_names:
        if last_count == 0:
            continue
        idx = last_count
        date = dfUS.index
        count = dfUS[name]
        date = date[30:]
        count = count[30:]
        color = hex_colors[idx % len(hex_colors)]
        fig.add_trace(
            go.Scatter(
                x=date, y=count, mode='lines+markers', name=f'{name} accumulated: last day = {count[-1]}', legendgroup=name,
                line=dict(color=color), marker=dict(color=color),
                hoverinfo='text',
                hovertext=[f'{name}, {t}={c}' for t, c in zip(date, count)]
            ),
            row=1, col=1
        )
        
        count = list(count)
        count_diff = np.array(count) - np.array([0]+count[:-1])
        fig.add_trace(
            go.Scatter(
                x=date, y=count_diff, mode='lines+markers', name=f'{name} daily new case: last day = {count_diff[-1]}', legendgroup=name,
                line=dict(color=color), marker=dict(color=color),
                hoverinfo='text',
                hovertext=[f'{name}, {t}={c}' for t, c in zip(date, count_diff)]
            ),
            row=2, col=1
        )
    
    fig.update_layout(title="Corona virus tracking: USA positive cases by states")
    fig.update_yaxes(title_text="Accumulative count", row=1, col=1)
    fig.update_yaxes(title_text="Daily count", row=2, col=1)
    
    plotly.offline.plot(fig, filename = 'corona_us.html', auto_open=False)

plot_usa_cases(df1)

In [4]:
def plot_usa_pos_neg_tracking(dfAll):
    df = dfAll
    states = df.state.unique()
    
    
    total_cols = 5
    total_rows = int(len(states)/total_cols) + 1
    
    fig = make_subplots(rows=total_rows, cols=total_cols, subplot_titles=states)
    bars = dict()
    for idx, state in enumerate(states):
        dfstate = df[df.state == state]
        dfstate = dfstate.sort_values(by='date')
        
        dates, positives, negatives = [],[],[]
        for idx, row in dfstate.iterrows():
            date, positive, negative = str(row.date), row.positive, row.negative
            if str(positive) == 'nan':
                positive = 0
            if str(negative)=='nan':
                negative = 0
            dates.append(pd.Timestamp(date))
            positives.append(positive)
            negatives.append(negative)
        data=[
            go.Bar(x=dates, y=positives, hovertext=['pos:'+str(int(p)) for p in positives]),
            go.Bar(x=dates, y=negatives, hovertext=['neg:'+str(int(n)) for n in negatives])
        ]
        
        cols = total_cols
        i, j = int(idx/cols)+1, idx%cols+1
        fig.add_trace(data[0], row=i, col=j)
        fig.add_trace(data[1], row=i, col=j)
        
    fig.update_layout(title="Corona virus tracking: USA positive & negative cases by states")
    fig.update_layout(barmode='stack', height=2000, showlegend=False)
    plotly.offline.plot(fig, filename = 'corona_states.html', auto_open=False)
    
plot_usa_pos_neg_tracking(df2)